In [66]:
%pip install pulp

     ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
     --- ------------------------------------ 1.2/14.3 MB 39.3 MB/s eta 0:00:01
     ---------- ----------------------------- 3.6/14.3 MB 38.2 MB/s eta 0:00:01
     ---------------- ----------------------- 5.8/14.3 MB 46.2 MB/s eta 0:00:01
     ----------------------- ---------------- 8.3/14.3 MB 44.0 MB/s eta 0:00:01
     ----------------------------- --------- 10.8/14.3 MB 50.4 MB/s eta 0:00:01
     ------------------------------------ -- 13.2/14.3 MB 50.4 MB/s eta 0:00:01
     --------------------------------------  14.2/14.3 MB 54.4 MB/s eta 0:00:01
     --------------------------------------- 14.3/14.3 MB 46.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
import pandas as pd

# Example data: each row represents an account with the financed amount, bad amount, and average amount
data = {
    'account_id': range(1, 11),
    'financed_amount': [1000, 1500, 800, 1200, 2000, 1300, 900, 1600, 1700, 1100],
    'bad_amount': [0, 150, 0, 120, 200, 0, 0, 160, 170, 0],
    'amt_avg': [1000, 1500, 800, 1200, 2000, 1300, 900, 1600, 1700, 1100]  # assuming amt_avg is the same for each row in this example
}

df = pd.DataFrame(data)

# Define the target bad_rate
target_bad_rate = 0.10  # for example, 10%

# Calculate the bad_rate for each account
df['bad_rate'] = df['bad_amount'] / df['amt_avg']

# Sort accounts by bad_rate, from lowest to highest
df_sorted = df.sort_values('bad_rate')

# Calculate the cumulative financed amount and the cumulative bad amount
df_sorted['cumulative_financed_amount'] = df_sorted['financed_amount'].cumsum()
df_sorted['cumulative_bad_amt'] = df_sorted['bad_amount'].cumsum()

# Now, we calculate the bad_rate based on the cumulative values
df_sorted['cumulative_bad_rate'] = df_sorted['cumulative_bad_amt'] / df_sorted['cumulative_financed_amount']

# Find the cutoff point where the cumulative bad_rate is just below the target
max_index = df_sorted[df_sorted['cumulative_bad_rate'] <= target_bad_rate].index.max()
cutoff_point = df_sorted.loc[max_index, 'account_id'] if pd.notna(max_index) else None

# Create a boolean mask for rows up to and including the cutoff index
mask = df_sorted.index <= max_index if cutoff_point else [False] * len(df_sorted)

# Use the boolean mask to select accounts to finance and reject
accounts_to_finance = df_sorted[mask]
accounts_to_reject = df_sorted[~mask]

print(f"Accounts to finance (up to cutoff point): \n{accounts_to_finance}")
print(f"Accounts to reject: \n{accounts_to_reject}")

# If you want to check the overall results
if not accounts_to_finance.empty:
    total_financed_amount = accounts_to_finance['financed_amount'].sum()
    final_bad_rate = accounts_to_finance['cumulative_bad_rate'].iloc[-1]
    print(f"Total financed amount: {total_financed_amount}")
    print(f"Final bad rate: {final_bad_rate}")


Accounts to finance (up to cutoff point): 
   account_id  financed_amount  bad_amount  amt_avg  bad_rate  \
0           1             1000           0     1000       0.0   
2           3              800           0      800       0.0   
5           6             1300           0     1300       0.0   
6           7              900           0      900       0.0   
9          10             1100           0     1100       0.0   
1           2             1500         150     1500       0.1   
3           4             1200         120     1200       0.1   
4           5             2000         200     2000       0.1   
7           8             1600         160     1600       0.1   
8           9             1700         170     1700       0.1   

   cumulative_financed_amount  cumulative_bad_amt  cumulative_bad_rate  
0                        1000                   0             0.000000  
2                        1800                   0             0.000000  
5                     

In [64]:
import pandas as pd
import numpy as np

# Set a seed for reproducibility
np.random.seed(0)

# Generate a synthetic dataset
num_records = 1000  # Number of records in the dataset
data = {
    'financed_amount': np.random.uniform(500, 10000, num_records),
    'bad_amount': np.random.uniform(0, 500, num_records),
    'amt_avg': np.random.uniform(200, 1500, num_records),
    'score1': np.random.randint(0, 5, num_records),  # Score between 0 and 4
    'score2': np.random.randint(0, 10, num_records)  # Score between 0 and 9
}

df = pd.DataFrame(data)

# Add some noise to the scores so that they're not uniformly distributed
for _ in range(100):  # Add noise
    idx = np.random.choice(df.index)
    df.at[idx, 'score1'] += np.random.choice([-1, 1])
    df.at[idx, 'score2'] += np.random.choice([-1, 1])

# Make sure scores are within the original bounds after noise
df['score1'] = df['score1'].clip(lower=0, upper=4)
df['score2'] = df['score2'].clip(lower=0, upper=9)

# Show the first few rows of the DataFrame
print(df.head())


   financed_amount  bad_amount      amt_avg  score1  score2
0      5713.728287  296.440135  1254.974012       1       9
1      7294.298981    5.031848   818.909182       4       6
2      6226.252073  237.913098   880.102787       0       5
3      5676.390238  354.385195   525.676762       1       1
4      4524.720594   21.987716   986.555922       3       6


In [88]:
import pandas as pd
import numpy as np
import pulp

# Seed for reproducibility
np.random.seed(0)

# Generate a synthetic dataset
num_records = 1000
data = {
    'financed_amount': np.random.uniform(500, 10000, num_records),
    'bad_amount': np.random.uniform(0, 500, num_records),
    'amt_avg': np.random.uniform(200, 1500, num_records),
    'score1': np.random.randint(0, 10, num_records),  # Score between 0 and 4
    'score2': np.random.randint(0, 10, num_records)  # Score between 0 and 9
}
df = pd.DataFrame(data)

# Add some noise to the scores
for _ in range(100):
    idx = np.random.choice(df.index)
    df.at[idx, 'score1'] += np.random.choice([-1, 1])
    df.at[idx, 'score2'] += np.random.choice([-1, 1])

target_financed_amount = 100000  # For example

# Define the problem
prob = pulp.LpProblem("MinimizeBadRate", pulp.LpMinimize)

# Decision variables
decision_vars = {}
for i, row in df.iterrows():
    for score1_cutoff in range(1, 11):  # Assuming you want cutoffs 1, 2, 3 for score1
        for score2_cutoff in range(1, 11):  # And 3, 4, 5 for score2
            # Variable creation logic based on score cutoffs
            decision_vars[(score1_cutoff, score2_cutoff, i)] = pulp.LpVariable(
                f"Finance_{score1_cutoff}_{score2_cutoff}_{i}", 0, 1, cat='Binary')

# Objective function: minimize the bad rate
bad_rate_var = pulp.LpVariable("BadRate", lowBound=0)
prob += bad_rate_var

# Constraints
# Ensure bad_rate_var represents the bad rate correctly
for i in df.index:
    prob += (
        bad_rate_var * (df.loc[i, 'amt_avg'] + 1e-6) >= pulp.lpSum([
            df.loc[i, 'bad_amount'] * decision_vars[(score1_cutoff, score2_cutoff, i)]
            for score1_cutoff in range(1, 11)
            for score2_cutoff in range(3, 11)
        ])
    ), f"BadRateConstraint_{i}"

# Ensure the target financed amount is met
prob += pulp.lpSum([
    df.loc[i, 'financed_amount'] * decision_vars[(score1_cutoff, score2_cutoff, i)]
    for i in df.index
    for score1_cutoff in range(1, 4)
    for score2_cutoff in range(3, 6)
]) >= target_financed_amount, "TargetFinancedAmount"

# Ensure a single decision per account
for i in df.index:
    prob += pulp.lpSum([
        decision_vars[(score1_cutoff, score2_cutoff, i)]
        for score1_cutoff in range(1, 11)
        for score2_cutoff in range(1, 11)
    ]) == 1, f"SingleDecisionRule_{i}"

# Solve the problem
prob.solve()

# Initialize a dictionary to store counts for score1 and the highest score2
optimal_decisions = {}

for var_key, variable in decision_vars.items():
    if pulp.value(variable) == 1:
        score1_cutoff = var_key[0]
        score2_cutoff = var_key[1]
        # Use a tuple (score1, score2) as the key
        decision_key = (score1_cutoff, score2_cutoff)
        
        # Initialize or increment the count for this decision
        if decision_key not in optimal_decisions:
            optimal_decisions[decision_key] = 1
        else:
            optimal_decisions[decision_key] += 1

# Sort the decisions by score1 and score2
sorted_decisions = sorted(optimal_decisions.items(), key=lambda item: (item[0][0], item[0][1]))

# Print the optimal decisions
for (score1_cutoff, score2_cutoff), count in sorted_decisions:
    print(f"Decisions Score1 >= {score1_cutoff} and Score2 > {score2_cutoff}: {count}")


# Initialize summary variables
approved_financed_amount = 0
approved_bad_amount = 0
rejected_financed_amount = 0
rejected_bad_amount = 0

# Iterate through the dataset to categorize each record
for i in df.index:
    record_approved = any(
        pulp.value(decision_vars[(score1_cutoff, score2_cutoff, i)]) == 1
        for score1_cutoff in range(1, 11)  # corrected range
        for score2_cutoff in range(1, 11)  # corrected range
    )
    if record_approved:
        approved_financed_amount += df.at[i, 'financed_amount']
        approved_bad_amount += df.at[i, 'bad_amount']
    else:
        rejected_financed_amount += df.at[i, 'financed_amount']
        rejected_bad_amount += df.at[i, 'bad_amount']
        
# Calculate bad rates
approved_bad_rate = approved_bad_amount / approved_financed_amount if approved_financed_amount > 0 else 0
rejected_bad_rate = rejected_bad_amount / rejected_financed_amount if rejected_financed_amount > 0 else 0

# Calculate the final decision impact
final_financed_amount = approved_financed_amount
final_bad_rate = approved_bad_amount / final_financed_amount if final_financed_amount > 0 else 0

# Print the summary
print(f"Approved Summary:")
print(f"  Total Financed Amount: {approved_financed_amount}")
print(f"  Total Bad Amount: {approved_bad_amount}")
print(f"  Bad Rate: {approved_bad_rate:.2%}")

print(f"\nRejected Summary:")
print(f"  Total Financed Amount: {rejected_financed_amount}")
print(f"  Total Bad Amount: {rejected_bad_amount}")
print(f"  Bad Rate: {rejected_bad_rate:.2%}")

print(f"\nFinal Decision Summary:")
print(f"  Total Financed Amount: {final_financed_amount}")
print(f"  Bad Rate: {final_bad_rate:.2%}")


Decisions Score1 >= 1 and Score2 > 1: 50
Decisions Score1 >= 1 and Score2 > 2: 43
Decisions Score1 >= 1 and Score2 > 3: 2
Decisions Score1 >= 1 and Score2 > 4: 2
Decisions Score1 >= 1 and Score2 > 5: 2
Decisions Score1 >= 2 and Score2 > 1: 45
Decisions Score1 >= 2 and Score2 > 2: 53
Decisions Score1 >= 2 and Score2 > 3: 2
Decisions Score1 >= 2 and Score2 > 4: 2
Decisions Score1 >= 2 and Score2 > 5: 1
Decisions Score1 >= 3 and Score2 > 1: 56
Decisions Score1 >= 3 and Score2 > 2: 61
Decisions Score1 >= 3 and Score2 > 3: 1
Decisions Score1 >= 3 and Score2 > 4: 2
Decisions Score1 >= 3 and Score2 > 5: 1
Decisions Score1 >= 4 and Score2 > 1: 49
Decisions Score1 >= 4 and Score2 > 2: 49
Decisions Score1 >= 5 and Score2 > 1: 48
Decisions Score1 >= 5 and Score2 > 2: 62
Decisions Score1 >= 6 and Score2 > 1: 43
Decisions Score1 >= 6 and Score2 > 2: 47
Decisions Score1 >= 7 and Score2 > 1: 50
Decisions Score1 >= 7 and Score2 > 2: 56
Decisions Score1 >= 8 and Score2 > 1: 47
Decisions Score1 >= 8 and